In [4]:
from six.moves import cPickle
import cv2
import numpy as np
from scipy.signal import convolve2d
import theano
import gym
from gym import wrappers
import theano
import theano.tensor as T
import lasagne

In [17]:
from lasagne.layers.dnn import Conv3DDNNLayer
from lasagne.layers.dnn import Pool3DDNNLayer

In [2]:
def _process_frame42(frame):
    frame = frame[34:34+160, :160]
    # Resize by half, then down to 42x42 (essentially mipmapping). If
    # we resize directly we lose pixels that, when mapped to 42x42,
    # aren't close enough to the pixel boundary.
    frame = cv2.resize(frame, (80, 80))
    frame = cv2.resize(frame, (42, 42))
    frame = frame.mean(2)
    frame = frame.astype(np.float32)
    frame *= (1.0 / 255.0)
    frame = np.reshape(frame, [42, 42, 1])
    return frame

In [5]:
env = gym.make("Skiing-v0")

[2017-02-06 16:41:39,038] Making new env: Skiing-v0


In [62]:
dtensor = T.TensorType('float32' ,(False,)*5)

In [66]:
def CreateNeuralNetwork(num_filters=10, filter_size=5, width=10):
    input_var = dtensor('inputs')
    target_var = dtensor('targets')
    network = lasagne.layers.InputLayer(shape=(None, 1, 4, 42, 42), input_var=input_var)
    network = Conv3DDNNLayer(network, num_filters, filter_size)
    network = lasagne.layers.DenseLayer(network, width, filter_size)
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    prediction_fn = theano.function([input_var], test_prediction)
    return prediction_fn

In [64]:
np.array(observation_frame).shape

(4, 42, 42)

In [60]:
pred_fn = CreateNeuralNetwork()

In [61]:
pred_fn(np.array([[observation_frame]]).astype('float32'))

RuntimeError: Could not set tensorNd descriptor: CUDNN_STATUS_BAD_PARAMdim=5
Apply node that caused the error: GpuDnnConv3d{algo='small', inplace=False}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty.0, GpuDnnConvDesc{border_mode=(0, 0, 0), subsample=(1, 1, 1), conv_mode='cross', precision='float32'}.0, Constant{1.0}, Constant{0.0})
Toposort index: 20
Inputs types: [CudaNdarrayType(float32, 5D), CudaNdarrayType(float32, (False, True, False, False, False)), CudaNdarrayType(float32, 5D), <theano.gof.type.CDataType object at 0x7f4c53957090>, Scalar(float32), Scalar(float32)]
Inputs shapes: [(1, 1, 4, 42, 42), (10, 1, 5, 5, 5), (1, 10, 0, 38, 38), 'No shapes', (), ()]
Inputs strides: [(0, 0, 1764, 42, 1), (125, 0, 25, 5, 1), (0, 0, 1444, 38, 1), 'No strides', (), ()]
Inputs values: ['not shown', 'not shown', CudaNdarray([]), <PyCObject object at 0x7f4c53b72c88>, 1.0, 0.0]
Inputs name: ('image', 'kernel', 'output', 'descriptor', 'alpha', 'beta')

Outputs clients: [[GpuElemwise{Composite{(i

In [9]:
np.array(observation_frame).shape

(4, 42, 42)

In [7]:
observation = env.reset()
done = False
iteration, all_reward = 0, 0
observation_frame = []
while not done:
    #env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
    observation_frame += [_process_frame42(observation)[:, :, 0]]
    if len(observation_frame) > 4:
        observation_frame.pop(0)
    print(len(observation_frame))
    all_reward += reward
    # if iteration % 500 == 0:
    # print(str(iteration) + "... ")

    if all_reward < -1000 or iteration >= 8000:
            break

    iteration += 1

print "Reward: ", all_reward

1
2
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
Reward:  -1002.0
